[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Hawksight-AI/semantica/blob/main/cookbook/use_cases/finance/03_Earnings_Call_Analysis.ipynb)

# Earnings Call Transcript Analysis with Docling and Semantica

## 📄 Earnings Call Transcript PDF

**Example:** Download a single earnings call transcript PDF from [SEC EDGAR](https://www.sec.gov/edgar/searchedgar/companysearch.html) or company investor relations pages. Use one PDF file for this analysis.

---

## Overview

Extract insights from earnings call transcripts by building a knowledge graph with entity extraction, relationship mapping, and GraphRAG-powered Q&A.

**Workflow:** `PDF → Parse → Extract Entities/Relations → Build KG → GraphRAG → Answers`


## Installation

```bash
pip install semantica
```


In [ ]:
# Initialize Groq LLM provider
from semantica.llms import Groq
import os

groq_llm = Groq(
    model="llama-3.1-8b-instant",
    api_key=os.getenv("GROQ_API_KEY")
)

print(f"✓ Groq LLM initialized: {groq_llm.model}")


## Step 1: Parse PDF with Docling

Parse earnings call PDF and extract financial tables using DoclingParser.


In [ ]:
# Step 1: Parse PDF with Docling
from semantica.parse import DoclingParser

parser = DoclingParser(
    export_format="markdown",
    enable_ocr=False,
    table_extraction_mode="auto"
)

# Example parsed document (replace with actual PDF parsing)
parsed_doc = {
    "full_text": """Q1 2024 Earnings Call Transcript

Company: TechCorp Inc.
Date: January 25, 2024

Prepared Remarks:
Our revenue for Q1 2024 was $2.5 billion, representing 15% year-over-year growth. 
EPS was $1.25 per share. We expect Q2 revenue to be between $2.6 and $2.8 billion.

Q&A Session:
Analyst: What's your guidance for the full year?
CEO: We're maintaining our full-year guidance of $10.5 to $11 billion in revenue.""",
    "tables": [],
    "metadata": {"title": "Q1 2024 Earnings Call"}
}

print(f"✓ Document parsed: {parsed_doc['metadata'].get('title', 'Unknown')}")
print(f"  Text length: {len(parsed_doc['full_text'])} characters")


## Step 2: Normalize Text

Normalize extracted text using TextNormalizer for consistent processing.


In [ ]:
# Step 2: Normalize text with Semantica
from semantica.normalize import TextNormalizer

text_normalizer = TextNormalizer()
normalized_text = text_normalizer.normalize(
    parsed_doc["full_text"],
    case="lower",
    remove_extra_whitespace=True
)

print(f"✓ Text normalized: {len(normalized_text)} characters")


## Step 3: Extract Entities

Extract entities (organizations, people, financial terms) using NERExtractor with Groq LLM.


In [ ]:
# Step 3: Extract entities using NERExtractor with Groq
from semantica.semantic_extract import NERExtractor

ner = NERExtractor(
    method="llm",
    provider="groq",
    llm_model="llama-3.1-8b-instant",
    min_confidence=0.7
)

entities = ner.extract_entities(
    normalized_text,
    entity_types=["ORG", "PERSON", "MONEY", "DATE", "PERCENT"]
)

print(f"✓ Entities extracted: {len(entities)}")
if entities:
    print(f"  Sample: {entities[0].text} ({entities[0].label})")


## Step 4: Extract Financial Metrics

Extract financial metrics (money, percentages, dates) from text and tables.


In [ ]:
# Step 4: Extract financial metrics using NERExtractor
financial_entities = ner.extract_entities(
    normalized_text,
    entity_types=["MONEY", "PERCENT", "DATE"]
)

financial_metrics = {}
for entity in financial_entities:
    if entity.label == "MONEY":
        financial_metrics[entity.text] = entity.text

print(f"✓ Financial entities: {len(financial_entities)}")
print(f"  Financial metrics: {len(financial_metrics)}")


## Step 5: Extract Relationships

Extract relationships between entities using RelationExtractor with Groq LLM.


In [ ]:
# Step 5: Extract relationships using RelationExtractor with Groq LLM
from semantica.semantic_extract import RelationExtractor

relation_extractor = RelationExtractor(
    method="llm",
    confidence_threshold=0.6,
    relation_types=["HAS_REVENUE", "HAS_EPS", "STATES", "PROVIDES_GUIDANCE", "OPERATES_IN"]
)

relationships = relation_extractor.extract_relations(
    normalized_text,
    entities=entities,
    provider="groq",
    llm_model="llama-3.1-8b-instant"
)

print(f"✓ Relationships extracted: {len(relationships)}")
if relationships:
    rel = relationships[0]
    print(f"  Sample: {rel.subject.text} → {rel.predicate} → {rel.object.text}")


## Step 6: Extract RDF Triplets

Extract RDF triplets (subject-predicate-object) using TripletExtractor with Groq LLM.


In [ ]:
# Step 6: Extract RDF triplets using TripletExtractor with Groq LLM
from semantica.semantic_extract import TripletExtractor

triplet_extractor = TripletExtractor(
    method="llm",
    include_temporal=True,
    include_provenance=True
)

triplets = triplet_extractor.extract_triplets(
    normalized_text,
    entities=entities,
    relations=relationships,
    provider="groq",
    llm_model="llama-3.1-8b-instant"
)

validated_triplets = triplet_extractor.validate_triplets(triplets)

print(f"✓ RDF triplets extracted: {len(triplets)}")
if triplets:
    t = triplets[0]
    print(f"  Sample: {t.subject} → {t.predicate} → {t.object}")


## Step 7: Detect Conflicts

Detect conflicts in extracted entities and relationships using ConflictDetector.


In [ ]:
# Step 7: Detect conflicts in extracted entities and relationships
from semantica.conflicts import ConflictDetector, SourceTracker, SourceReference

source_tracker = SourceTracker()
conflict_detector = ConflictDetector(
    source_tracker=source_tracker,
    confidence_threshold=0.7
)

# Track sources for entities
for entity in entities:
    entity_id = getattr(entity, 'id', None) or getattr(entity, 'text', '')
    entity_name = getattr(entity, 'text', '')
    source_tracker.track_property_source(
        entity_id,
        'name',
        entity_name,
        source=SourceReference(
            source='earnings_call',
            timestamp='2024-Q1',
            metadata={'entity_type': getattr(entity, 'label', 'UNKNOWN')}
        )
    )

# Detect value conflicts
value_conflicts = conflict_detector.detect_value_conflicts(
    [{'id': getattr(e, 'id', ''), 'name': getattr(e, 'text', '')} for e in entities],
    property_name='name'
)

# Detect relationship conflicts
relationship_conflicts = conflict_detector.detect_relationship_conflicts(relationships)

print(f"✓ Conflicts detected")
print(f"  Value conflicts: {len(value_conflicts)}")
print(f"  Relationship conflicts: {len(relationship_conflicts)}")


## Step 8: Resolve Conflicts

Resolve detected conflicts using ConflictResolver with voting strategy.


In [ ]:
# Step 8: Resolve conflicts using ConflictResolver
from semantica.conflicts import ConflictResolver

conflict_resolver = ConflictResolver(
    default_strategy='voting',
    source_tracker=source_tracker
)

# Resolve value conflicts
resolved_entities = list(entities)
resolved_conflicts = []
for conflict in value_conflicts:
    resolution = conflict_resolver.resolve_conflict(conflict, strategy='voting')
    resolved_conflicts.append(resolution)

# Resolve relationship conflicts
resolved_relationships = list(relationships)
for conflict in relationship_conflicts:
    resolution = conflict_resolver.resolve_conflict(conflict, strategy='voting')
    resolved_conflicts.append(resolution)

print(f"✓ Conflicts resolved: {len(resolved_conflicts)}")


## Step 9: Deduplicate Entities

Detect and merge duplicate entities using DuplicateDetector and EntityMerger.


In [ ]:
# Step 9: Deduplicate entities using DuplicateDetector and EntityMerger
from semantica.deduplication import DuplicateDetector, EntityMerger

duplicate_detector = DuplicateDetector(
    similarity_threshold=0.8,
    confidence_threshold=0.7
)

# Convert entities to dict format
entity_dicts = []
for entity in resolved_entities:
    entity_dicts.append({
        'id': getattr(entity, 'id', ''),
        'name': getattr(entity, 'text', ''),
        'type': getattr(entity, 'label', 'UNKNOWN'),
        'confidence': getattr(entity, 'confidence', 1.0),
        'metadata': getattr(entity, 'metadata', {})
    })

# Detect duplicates
duplicates = duplicate_detector.detect_duplicates(entity_dicts)

# Merge duplicates
entity_merger = EntityMerger(preserve_provenance=True)
merge_operations = entity_merger.merge_duplicates(
    entity_dicts,
    strategy='keep_most_complete'
)

merged_entities = [op.merged_entity for op in merge_operations]

print(f"✓ Deduplication complete")
print(f"  Original entities: {len(entity_dicts)}")
print(f"  Merged entities: {len(merged_entities)}")
print(f"  Duplicates removed: {len(entity_dicts) - len(merged_entities)}")


## Step 10: Build Knowledge Graph

Build knowledge graph from cleaned entities, relationships, and triplets using GraphBuilder.


In [ ]:
# Step 10: Build knowledge graph from cleaned entities, resolved relationships, and triplets
from semantica.kg import GraphBuilder

graph_builder = GraphBuilder(
    merge_entities=True,
    entity_resolution_strategy="fuzzy"
)

# Convert triplets to relationships format
triplet_relationships = []
for triplet in triplets:
    triplet_relationships.append({
        "source": triplet.subject,
        "predicate": triplet.predicate,
        "target": triplet.object,
        "confidence": triplet.confidence,
        "metadata": triplet.metadata
    })

all_relationships = resolved_relationships + triplet_relationships

kg_data = {
    "entities": merged_entities,
    "relationships": all_relationships,
    "triplets": triplets,
    "metadata": {
        "source": "earnings_call_transcript",
        "financial_metrics": financial_metrics,
        "extraction_method": "Groq LLM"
    }
}

knowledge_graph = graph_builder.build(
    sources=[kg_data],
    merge_entities=True
)

print(f"✓ Knowledge graph built")
print(f"  Entities: {len(knowledge_graph.get('entities', []))}")
print(f"  Relationships: {len(knowledge_graph.get('relationships', []))}")


## Step 11: Analyze Knowledge Graph

Analyze graph structure using GraphAnalyzer (centrality, communities, connectivity).


In [ ]:
# Step 11: Analyze knowledge graph using GraphAnalyzer
from semantica.kg import GraphAnalyzer

graph_analyzer = GraphAnalyzer()
analysis = graph_analyzer.analyze_graph(knowledge_graph)
centrality = graph_analyzer.calculate_centrality(knowledge_graph, 'degree')
communities = graph_analyzer.detect_communities(knowledge_graph, algorithm='louvain')
connectivity = graph_analyzer.analyze_connectivity(knowledge_graph)
metrics = graph_analyzer.compute_metrics(knowledge_graph)

top_entities = []
if centrality and 'rankings' in centrality:
    top_entities = centrality['rankings'][:5]

num_communities = len(communities.get('communities', [])) if isinstance(communities, dict) else 0

print(f"✓ Graph analysis complete")
print(f"  Communities: {num_communities}")
print(f"  Top entities: {len(top_entities)}")


## Step 12: Build Context Graph

Build ContextGraph from knowledge graph for enhanced retrieval and GraphRAG.


In [ ]:
# Step 12: Build context graph for enhanced retrieval
from semantica.context import ContextGraph

context_graph = ContextGraph(
    extract_entities=True,
    extract_relationships=True
)

# Convert knowledge graph to context graph format
nodes = []
for entity in knowledge_graph.get('entities', []):
    nodes.append({
        "id": entity.get('id', entity.get('name', '')),
        "type": entity.get('type', 'entity'),
        "properties": {
            "content": entity.get('name', ''),
            "confidence": entity.get('confidence', 1.0),
            **entity.get('metadata', {})
        }
    })

edges = []
for rel in knowledge_graph.get('relationships', []):
    edges.append({
        "source_id": rel.get('source', ''),
        "target_id": rel.get('target', ''),
        "type": rel.get('predicate', 'related_to'),
        "weight": rel.get('confidence', 1.0)
    })

node_count = context_graph.add_nodes(nodes)
edge_count = context_graph.add_edges(edges)

print(f"✓ Context graph built")
print(f"  Nodes: {node_count}")
print(f"  Edges: {edge_count}")


## Step 13: Context Retrieval

Set up hybrid retrieval (vector + graph) using ContextRetriever for GraphRAG queries.


In [ ]:
# Step 13: Set up hybrid context retrieval and demonstrate GraphRAG
from semantica.vector_store import VectorStore
from semantica.context import ContextRetriever

# Initialize VectorStore
vector_store = VectorStore(backend="faiss")
vector_store.add(
    texts=[parsed_doc["full_text"]],
    metadata=[{"source": "earnings_call", "type": "transcript"}]
)

# Initialize ContextRetriever
context_retriever = ContextRetriever(
    knowledge_graph=context_graph,
    vector_store=vector_store,
    hybrid_alpha=0.6,
    use_graph_expansion=True,
    max_expansion_hops=2
)

# Retrieve context for financial queries
financial_queries = [
    "What was the company's revenue guidance?",
    "What were the key financial metrics discussed?"
]

retrieved_contexts = []
for query in financial_queries:
    results = context_retriever.retrieve(
        query=query,
        max_results=3,
        min_relevance_score=0.2
    )
    retrieved_contexts.append({
        "query": query,
        "results": results,
        "count": len(results)
    })

print(f"✓ Hybrid retrieval configured")
print(f"  Queries processed: {len(retrieved_contexts)}")


## Step 14: Entity Linking

Link entities across sources and assign URIs using EntityLinker.


In [ ]:
# Step 14: Link entities using EntityLinker
from semantica.context import EntityLinker

entity_linker = EntityLinker(knowledge_graph=knowledge_graph)

# Assign URIs to key entities
linked_entities = []
for entity in merged_entities[:10]:
    entity_id = entity.get('id', entity.get('name', ''))
    entity_name = entity.get('name', '')
    entity_type = entity.get('type', 'UNKNOWN')
    
    uri = entity_linker.assign_uri(
        entity_id=entity_id,
        text=entity_name,
        entity_type=entity_type
    )
    linked_entities.append({
        "entity_id": entity_id,
        "name": entity_name,
        "uri": uri,
        "type": entity_type
    })

# Build entity web
entity_web = entity_linker.build_entity_web()

print(f"✓ Entity linking complete")
print(f"  Entities linked: {len(linked_entities)}")
print(f"  Entity web nodes: {len(entity_web.get('nodes', []))}")


## Step 15: Agent Memory

Store and retrieve memories using AgentMemory with RAG integration.


In [ ]:
# Step 15: Store and retrieve memories using AgentMemory
from semantica.context import AgentMemory

agent_memory = AgentMemory(
    vector_store=vector_store,
    knowledge_graph=knowledge_graph,
    retention_days=30
)

# Store earnings call memories
memory_ids = []
memory_contents = [
    f"Earnings call transcript: {parsed_doc['metadata'].get('title', 'Q1 2024')}",
    f"Financial metrics extracted: {len(financial_metrics)} metrics",
    f"Key entities identified: {len(merged_entities)} entities"
]

for content in memory_contents:
    memory_id = agent_memory.store(
        content=content,
        metadata={"source": "earnings_call", "type": "transcript_analysis"},
        extract_entities=True,
        extract_relationships=True
    )
    memory_ids.append(memory_id)

# Retrieve memories
financial_memories = agent_memory.retrieve(
    query="financial metrics and earnings",
    max_results=5
)

memory_stats = agent_memory.get_statistics()

print(f"✓ Agent memory configured")
print(f"  Memories stored: {len(memory_ids)}")
print(f"  Total memories: {memory_stats.get('total_memories', 0)}")


## Step 16: Agent Context

Unified context management with AgentContext (auto-detects RAG vs GraphRAG).


In [ ]:
# Step 16: High-level context management with AgentContext
from semantica.context import AgentContext

agent_context = AgentContext(
    vector_store=vector_store,
    knowledge_graph=context_graph,
    use_graph_expansion=True,
    max_expansion_hops=2,
    hybrid_alpha=0.6,
    retention_days=30
)

# Store content with auto-extraction
memory_id = agent_context.store(
    content=parsed_doc["full_text"][:1000],
    metadata={"source": "earnings_call", "date": "2024-Q1"},
    extract_entities=True,
    extract_relationships=True,
    link_entities=True
)

# Retrieve with auto-detected GraphRAG
graphrag_results = agent_context.retrieve(
    query="What was discussed about revenue growth?",
    max_results=5,
    expand_graph=True,
    include_entities=True
)

context_stats = agent_context.stats()

print(f"✓ AgentContext configured")
print(f"  Memory stored: {memory_id}")
print(f"  GraphRAG results: {len(graphrag_results)}")
print(f"  Total memories: {context_stats.get('total_memories', 0)}")


## Step 17: Answer Generation

Generate answers to financial questions using Groq LLM with retrieved context and knowledge graph.


## Step 18: Export Results

Export knowledge graph and analysis results to JSON and RDF formats.


In [ ]:
# Step 17: Generate answers using Groq LLM from semantica.llms module
financial_questions = [
    "What were the key financial metrics discussed in the earnings call?",
    "What guidance did management provide for future quarters?"
]

generated_answers = []
for question in financial_questions:
    # Retrieve relevant context
    context_results = context_retriever.retrieve(
        query=question,
        max_results=3,
        min_relevance_score=0.2
    )
    
    # Build context from retrieved results
    context_text = "\n\n".join([
        f"Context {i+1}: {result.get('content', result.get('text', ''))}"
        for i, result in enumerate(context_results[:3])
    ])
    
    # Extract relevant entities
    relevant_entities = [
        entity.get('name', '') for entity in knowledge_graph.get('entities', [])[:10]
    ]
    entities_text = ", ".join(relevant_entities[:5]) if relevant_entities else "N/A"
    
    # Build prompt
    prompt = f"""Based on the following earnings call transcript context and knowledge graph, answer the question.

Context from transcript:
{context_text[:1000]}

Key entities identified: {entities_text}

Question: {question}

Provide a comprehensive answer based on the context provided. If information is not available in the context, state that clearly.

Answer:"""
    
    # Generate answer using Groq LLM
    try:
        answer = groq_llm.generate(
            prompt,
            temperature=0.7,
            max_tokens=500
        )
        generated_answers.append({
            "question": question,
            "answer": answer,
            "context_sources": len(context_results),
            "model": groq_llm.model
        })
    except Exception as e:
        generated_answers.append({
            "question": question,
            "answer": f"Error generating answer: {str(e)}",
            "context_sources": len(context_results),
            "model": groq_llm.model
        })

print(f"✓ Answer generation complete using Groq LLM")
print(f"  LLM Provider: Groq ({groq_llm.model})")
print(f"  Questions answered: {len(generated_answers)}")
if generated_answers:
    print(f"  Sample question: '{generated_answers[0]['question']}'")


In [ ]:
# Step 18: Export structured outputs including triplets
from semantica.export import JSONExporter, RDFExporter

json_exporter = JSONExporter()
rdf_exporter = RDFExporter()

# Export knowledge graph to JSON
kg_json = json_exporter.export(knowledge_graph, format="json")

# Export knowledge graph to RDF (Turtle format)
rdf_output = rdf_exporter.export_to_rdf(knowledge_graph, format="turtle")

# Create analysis summary
analysis_summary = {
    "financial_metrics": financial_metrics,
    "extraction_stats": {
        "entities": len(entities),
        "relationships": len(relationships),
        "triplets": len(triplets),
        "provider": f"Groq LLM (semantica.llms module) - {groq_llm.model}"
    },
    "conflict_resolution": {
        "conflicts_detected": len(value_conflicts) + len(relationship_conflicts),
        "conflicts_resolved": len(resolved_conflicts),
        "strategy": "voting"
    },
    "deduplication": {
        "original_entities": len(entity_dicts),
        "duplicates_detected": len(duplicates),
        "merged_entities": len(merged_entities),
        "strategy": "keep_most_complete"
    },
    "knowledge_graph": {
        "entities": len(knowledge_graph.get('entities', [])),
        "relationships": len(knowledge_graph.get('relationships', []))
    },
    "graph_analytics": {
        "metrics": metrics,
        "communities": num_communities,
        "top_entities": top_entities[:5] if top_entities else []
    },
    "context_graph": {
        "nodes": len(context_graph.nodes),
        "edges": len(context_graph.edges)
    },
    "context_retrieval": {
        "queries_processed": len(retrieved_contexts),
        "total_results": sum(c["count"] for c in retrieved_contexts)
    },
    "entity_linking": {
        "entities_linked": len(linked_entities),
        "entity_web_nodes": len(entity_web.get('nodes', [])),
        "entity_web_edges": len(entity_web.get('edges', []))
    },
    "agent_memory": {
        "memories_stored": len(memory_ids),
        "total_memories": memory_stats.get('total_memories', 0)
    },
    "agent_context": {
        "graphrag_results": len(graphrag_results),
        "total_memories": context_stats.get('total_memories', 0)
    },
    "answer_generation": {
        "questions_answered": len(generated_answers),
        "llm_provider": "Groq",
        "llm_model": groq_llm.model,
        "answers": [
            {
                "question": ans["question"],
                "answer_length": len(ans["answer"]),
                "context_sources": ans["context_sources"]
            }
            for ans in generated_answers
        ]
    }
}

print(f"✓ Export complete")
print(f"  Analysis summary: {len(analysis_summary)} sections")
print(f"  Knowledge graph (JSON): {len(kg_json) if isinstance(kg_json, dict) else 0} items")
print(f"  RDF (Turtle): {len(rdf_output)} characters")
print(f"  LLM answers generated: {len(generated_answers)}")
print(f"  LLM provider: Groq ({groq_llm.model})")
